In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import TimeoutException

import json
import time
import os
import re

In [79]:
def remplirjsonUtilisateur(url, donnees, utilisateur, contenu):   
    # Ajouter les nouvelles données
    if url not in contenu:
        if(donnees["chaine"] != None):
            contenu[url] = {
                "chaine": donnees["chaine"],
                "users": {
                    utilisateur: {
                        "date": donnees["date"],
                        "heure": donnees["heure"]
                    }
                }
            }

    else: 
        if "users" not in contenu[url]:
            contenu[url]["users"] = {}
        contenu[url]["users"][utilisateur] = {
            "date": donnees["date"],
            "heure": donnees["heure"]
        }
    return contenu

In [80]:
def format_date(date):
    datechaine = date.split(" ")
    dateFinale = ""
    if(len(datechaine[0]) == 1):
        dateFinale = "0"
    dateFinale += datechaine[0] + "/"
    
    month = ["janv.", "févr.", "mars", "avr.", "mai", "juin", "juil.", "août", "sept.", "oct.", "nov.", "déc."]
    monthTemp = ""
    if(datechaine[1] in month):
        monthTemp += str(month.index(datechaine[1])+1)
    if(len(monthTemp) == 1):
        monthTemp = "0" + monthTemp
    dateFinale += monthTemp + "/"
    dateFinale += datechaine[2]
    return dateFinale

In [81]:
html_content = "watch-historyLoric.html"

soup = BeautifulSoup(open(html_content, encoding="utf-8"), "html.parser")

blocks = soup.find_all("div", class_="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1")

In [82]:
videos = []

for block in blocks:
    text_content = block.get_text(separator=" ", strip=True)
    # Extraire le lien de la vidéo
    video_link_tag = block.find("a", href=True)
    video_link = video_link_tag['href'] if video_link_tag else None

    # Extraire le nom de la chaîne
    channel_link_tag = block.find_all("a", href=True)[1] if len(block.find_all("a", href=True)) > 1 else None
    channel_name = channel_link_tag.text if channel_link_tag else None

    # Extraire la date et l'heure
    match = re.search(r"(\d{1,2} \w+. \d{4}, \d{1,2}:\d{1,2}:\d{1,2})", text_content)
    date, heure = match.group(1).split(",") if match else (None, None)
    date = format_date(date) if date else None
    # Stocker les données
    videos.append({
        "lien_video": video_link,
        "chaine": channel_name,
        "date": date,
        "heure": heure
    })

In [84]:
if os.path.exists("metadataCombo.json"):
    # Lire les données existantes
    with open("metadataCombo.json", 'r', encoding='utf-8') as fichier:
        try:
            contenu = json.load(fichier)
        except json.JSONDecodeError:
            contenu = {}  # Initialiser comme une liste vide si le fichier est vide ou corrompu
else:
    contenu = {}  # Si le fichier n'existe pas, initialiser une liste vide

print(len(contenu))

for video in videos:
    contenu = remplirjsonUtilisateur(video["lien_video"], video, "Loric", contenu)

print(len(contenu))

# Écrire les données mises à jour dans le fichier
with open("metadataCombo.json", 'w', encoding='utf-8') as fichier:
    json.dump(contenu, fichier, indent=4, ensure_ascii=False)

50917
87961


# Youtube scraping

In [2]:
def countryChannel(url):
    options = Options()
    driverCountry = webdriver.Firefox(
        options=options
    )
    
    driverCountry.install_addon('ublock_origin-1.61.2.xpi', temporary=True)
    # visit the target page in the controlled browser
    driverCountry.get(url)
    
    
    wait = WebDriverWait(driverCountry, 9)
    try:
        # might need a sleep timer here to wait for the consent page to load
        time.sleep(3)
        if 'consent.youtube.com' in driverCountry.current_url: # THIS IS THE CHECK
            driverCountry.find_element("xpath",'//button[@aria-label="Accept all"]').click() # THIS ACCEPTS THE COOKIE CONSENT FORM
        #wait.until(EC.element_to_be_clickable((By.XPATH, '//yt-formatted-string[@class="style-scope ytd-channel-name"]')))
    except:
        print('Failed to load consent page')
        
    # wait for YouTube to load the page data
    WebDriverWait(driverCountry, 60).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.dynamic-text-view-model-wiz__h1'))
    )
    
    try:
        driverCountry.find_element("xpath", '//button[@class="truncated-text-wiz__absolute-button"]').click()
        wait.until(EC.element_to_be_clickable((By.XPATH, '//ytd-engagement-panel-section-list-renderer[@class="style-scope ytd-popup-container"]')))
        driverCountry.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        country = driverCountry.find_element("xpath", "//table[@class='style-scope ytd-about-channel-renderer']/tbody/tr[8]/td[2]").text
    except:
        print('Failed to load popup')
        
    driverCountry.quit()
    return country if country else None


In [4]:
def metaDta(url):
    # initialize a web driver instance to control a Chrome window
    # in headless mode
    options = Options()
    options.add_argument('--headless')

    driver = webdriver.Firefox(
        options=options
    )
    
    driver.install_addon('ublock_origin-1.61.2.xpi', temporary=True)
    # visit the target page in the controlled browser
    driver.get(url)
    print(url)
    
    try:
    # wait up to 15 seconds for the consent dialog to show up
        consent_overlay = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, 'dialog'))
        )
        # wait for YouTube to load the page data

        # select the consent option buttons
        consent_buttons = consent_overlay.find_elements(By.CSS_SELECTOR, '.eom-buttons button.yt-spec-button-shape-next')
        if len(consent_buttons) > 1:
            # retrieve and click the 'Accept all' button
            accept_all_button = consent_buttons[1]
            accept_all_button.click()
    
            # wait for YouTube to load the page data
        WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.ytd-watch-metadata'))
        )

        # initialize the dictionary that will contain
        # the data scraped from the YouTube page
        video = {}
        try :
            # scrape the video duration
            duration = driver.find_element("xpath", "//span[@class='ytp-time-duration']")
            duree = duration.text.split(":")
            
            if len(duree) == 2:  # Format 'm:ss'
                minutes, secondes = duree
                heures = "00"
            elif len(duree) == 3:  # Format 'hh:mm:ss'
                heures, minutes, secondes = duree
            dureeFinale = f"{int(heures):02}:{int(minutes):02}:{int(secondes):02}"
                
            
        except:
            print("Erreur de durée sur " + url)
            dureeFinale = None
            
        # # scrape the channel info attributes
        # try :
        #     channel_element = driver \
        #         .find_element(By.ID, 'owner')
        # except:
        #     print("Erreur de chaine sur " + url)
                            
        # time.sleep(1)
        # country = None
        # try :
        #     channel_url = channel_element \
        #             .find_element(By.CSS_SELECTOR, 'a.yt-simple-endpoint') \
        #             .get_attribute('href')
        #     country = countryChannel(channel_url)
        # except:
        #     print("Erreur de pays sur " + url)
        # time.sleep(1)
        
        try:
            elem = driver.find_element("xpath", "//div[@class='watch-main-col']/meta[@itemprop='genre']")
        except:
            print("Erreur de genre sur " + url)
        
        video['genre'] = elem.get_attribute("content")
        video['duree'] = dureeFinale
        # if country:
        #     video['pays'] = country
        # else:
        #     video['pays'] = None
        
        
        
        driver.quit()
    
    except TimeoutException:
        print('Cookie modal missing sur ' + url)
        
    return video
    

In [ ]:
with open("metadataCombo.json", 'r', encoding='utf-8') as fichier:
    contenu = json.load(fichier)

videos = list(contenu.keys())

i = 190
for video in videos[i:]:
    try :
        dataupdate = metaDta(video)
        contenu[video].update(dataupdate)
        i += 1
        if(i % 10 == 0):
            with open("metadataCombo.json", 'w', encoding='utf-8') as fichier:
                json.dump(contenu, fichier, indent=4, ensure_ascii=False)
            print("Sauvegarde à l'index " + str(i))
    except:
        print("Erreur sur " + video)
        continue

In [85]:
def nettoyer_json(path, champs_requis):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Le fichier JSON à l'emplacement {path} n'existe pas.")
    
    # Charger les données du fichier JSON
    with open(path, 'r', encoding='utf-8') as fichier:
        try:
            json_data = json.load(fichier)
        except json.JSONDecodeError:
            raise ValueError("Le fichier JSON est corrompu ou vide.")
    
    # Nettoyer les données
    json_nettoye = {
        url: video_data
        for url, video_data in json_data.items()
        if all(champ in video_data for champ in champs_requis)
    }
    
    # Écrire les données nettoyées dans le fichier
    with open(path, 'w', encoding='utf-8') as fichier:
        json.dump(json_nettoye, fichier, indent=4, ensure_ascii=False)
    
    return json_nettoye

In [86]:
champs_requis = ["genre", "duree", "pays"]

json_nettoye = nettoyer_json("metadata.json", champs_requis)

Données nettoyées avec succès :
{
    "https://www.youtube.com/watch?v=h4fwCvsFE8Y": {
        "channel_name": "JOYCA",
        "users": {
            "Angéline": {
                "date": "02/12/2024",
                "heure": " 21:12:46"
            }
        },
        "genre": "Entertainment",
        "duree": "00:59:21",
        "pays": "France"
    },
    "https://www.youtube.com/watch?v=Sqpm5Xehdug": {
        "channel_name": "Nintendo France",
        "users": {
            "Angéline": {
                "date": "29/11/2024",
                "heure": " 22:32:14"
            }
        },
        "genre": "Gaming",
        "duree": "00:02:51",
        "pays": null
    },
    "https://www.youtube.com/watch?v=omgSWqwVTjY": {
        "channel_name": "Riot Games Music",
        "users": {
            "Angéline": {
                "date": "29/11/2024",
                "heure": " 19:40:45"
            }
        },
        "genre": "Music",
        "duree": "00:03:50",
        "pays": "U

In [87]:
def completer_pays_from_file(json_path):
    """
    Complète les pays manquants dans un fichier JSON basé sur le même `channel_name`.
    
    :param json_path: Chemin du fichier JSON contenant les données des vidéos.
    """
    # Charger les données depuis le fichier JSON
    with open(json_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)

    # Dictionnaire temporaire pour stocker le pays par channel_name
    pays_par_channel = {}

    # Première passe : collecter les pays disponibles par `channel_name`
    for url, data in json_data.items():
        channel_name = data.get("channel_name")
        pays = data.get("pays")
        if channel_name and pays:
            pays_par_channel[channel_name] = pays

    # Deuxième passe : compléter les pays manquants
    for url, data in json_data.items():
        channel_name = data.get("channel_name")
        if channel_name and data.get("pays") is None:
            # Compléter le pays si disponible pour ce channel_name
            data["pays"] = pays_par_channel.get(channel_name)

    # Sauvegarder les données mises à jour dans le même fichier
    with open(json_path, 'w', encoding='utf-8') as file:
        json.dump(json_data, file, indent=4, ensure_ascii=False)

In [88]:
completer_pays_from_file("metadata.json")

Fichier JSON mis à jour avec succès : metadata.json
